In [1]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import open3d as o3d
from utils import read_files

In [2]:
basedir = 'datasets/3rdfloor'
depth_dir = os.path.join(basedir, 'depth')
depth_midas_dir = os.path.join(basedir, 'depth_midas')
image_dir = os.path.join(basedir, 'images')
imnames = [
 'frame000702.jpg',
 'frame000609.jpg',
 'frame000144.jpg',
 'frame001101.jpg',
 'frame001446.jpg']

In [20]:
i+=1
imname = imnames[i]
image = cv2.imread(os.path.join(image_dir, imname))
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# mine
depth = cv2.imread(os.path.join(depth_dir,
                                imname.replace('.jpg', '.png')), -1) / 1000.
# midas
inv_depth = cv2.imread(os.path.join(depth_midas_dir,
                                imname.replace('.jpg', '.png')), -1)
depth_midas = 1. / inv_depth

# plt.rcParams["figure.figsize"] = (12,12)
# plt.subplot(1, 3, 1)
# plt.imshow(image)
# plt.subplot(1, 3, 2)
# plt.imshow(depth)
# plt.subplot(1, 3, 3)
# plt.imshow(depth_midas)

IndexError: list index out of range

In [19]:
image_o3d = o3d.geometry.Image(image)

intrinsics = o3d.camera.PinholeCameraIntrinsic(depth.shape[1],
                                               depth.shape[0], 
                                               1666.82,
                                               1666.82,
                                               depth.shape[1] / 2,
                                               depth.shape[0] / 2)
########
depth_o3d = o3d.geometry.Image(depth.astype(np.float32))
rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(image_o3d,
                                                          depth_o3d, 
                                                          convert_rgb_to_intensity=False)        
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsics)
########
depth_o3d2 = o3d.geometry.Image((depth_midas*5500).astype(np.float32))
rgbd2 = o3d.geometry.RGBDImage.create_from_color_and_depth(image_o3d,
                                                          depth_o3d2, 
                                                          convert_rgb_to_intensity=False)        
pcd2 = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd2, intrinsics)
pcd2.translate([0.004,0,0])
o3d.visualization.draw_geometries([pcd, pcd2])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


### visualize trajectory

In [9]:
poses_dir = 'datasets/3rdfloor/poses'
poses = sorted(read_files(poses_dir, ext=('.txt'))[poses_dir])

coords = []

for i in range(len(poses)):
    if i%6==0:
        mtx = np.genfromtxt(os.path.join(poses_dir, poses[i]), delimiter=' ')
        coord = o3d.geometry.TriangleMesh.create_coordinate_frame()
        coord.transform(mtx)
        coords.append(coord)

o3d.visualization.draw_geometries(coords)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
